In [2]:
import cv2
import numpy as np
import time # -- 프레임 계산을 위해 사용


In [5]:
import serial
#vedio_path = './video.mp4' #-- 사용할 영상 경로
min_confidence = 0.4

#serial = serial.Serial(
#    port = 'COM3',
#    baudrate = 9600)

def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.8, fy=0.8)
    height, width, channels = img.shape
    #cv2.imshow("Original Image", img)

    #-- 창 크기 설정
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    #-- 탐지한 객체의 클래스 예측 
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > min_confidence:
                # 탐지한 객체 박싱
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
    font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]], confidences[i]*100)
            #serial.write(classes[class_ids[i]].encode())
            time.sleep(0.1)
            print(i, label)
            color = colors[i] #-- 경계 상자 컬러 설정 / 단일 생상 사용시 (255,255,255)사용(B,G,R)
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    cv2.imshow("YOLO test", img)
    
#-- yolo 포맷 및 클래스명 불러오기
model_file = './yolov3-tiny.weights' #-- 본인 개발 환경에 맞게 변경할 것
config_file = './yolov3-tiny.cfg' #-- 본인 개발 환경에 맞게 변경할 것
net = cv2.dnn.readNet(model_file, config_file)

#-- GPU 사용
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

#-- 클래스(names파일) 오픈 / 본인 개발 환경에 맞게 변경할 것
classes = []
with open("./coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

#-- 비디오 활성화
cap = cv2.VideoCapture(0) #-- 웹캠 사용시 vedio_path를 0 으로 변경
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    
    #cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
    #cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
    ret, frame = cap.read()
    print(frame)
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)
    #-- q 입력시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

[[[180 178 178]
  [178 177 177]
  [177 177 177]
  ...
  [137 145 150]
  [136 144 150]
  [137 145 151]]

 [[180 178 178]
  [178 177 177]
  [177 177 177]
  ...
  [136 144 148]
  [135 143 147]
  [135 143 147]]

 [[180 178 178]
  [178 177 177]
  [177 177 177]
  ...
  [136 145 146]
  [135 144 145]
  [135 144 145]]

 ...

 [[ 44  59  71]
  [ 44  59  70]
  [ 44  60  70]
  ...
  [160 163 160]
  [155 158 155]
  [160 163 160]]

 [[ 47  63  73]
  [ 46  62  72]
  [ 46  62  72]
  ...
  [119 122 126]
  [122 124 126]
  [121 123 125]]

 [[ 48  64  74]
  [ 48  64  74]
  [ 47  63  73]
  ...
  [114 115 123]
  [115 117 121]
  [113 115 118]]]
0 person: 49.74
=== A frame took 0.796 seconds
[[[181 178 173]
  [180 178 173]
  [179 178 173]
  ...
  [135 142 149]
  [134 141 147]
  [134 141 147]]

 [[179 178 173]
  [179 178 173]
  [178 177 172]
  ...
  [134 143 147]
  [134 142 146]
  [134 142 146]]

 [[179 177 175]
  [178 176 173]
  [178 177 172]
  ...
  [134 142 146]
  [134 142 146]
  [134 142 146]]

 ...

 [[ 4

=== A frame took 0.189 seconds
[[[176 178 178]
  [175 177 177]
  [175 177 177]
  ...
  [140 144 147]
  [141 142 147]
  [141 142 147]]

 [[176 178 178]
  [175 177 177]
  [176 178 178]
  ...
  [139 144 148]
  [139 143 147]
  [139 143 147]]

 [[176 179 176]
  [176 179 176]
  [176 179 176]
  ...
  [139 143 147]
  [138 141 146]
  [138 141 146]]

 ...

 [[ 44  62  72]
  [ 44  62  72]
  [ 44  62  72]
  ...
  [157 163 159]
  [155 161 156]
  [158 163 158]]

 [[ 43  64  73]
  [ 43  64  73]
  [ 43  64  73]
  ...
  [124 128 137]
  [127 130 137]
  [122 125 131]]

 [[ 44  65  72]
  [ 44  65  74]
  [ 44  64  76]
  ...
  [113 114 131]
  [116 117 131]
  [113 115 128]]]
0 person: 49.62
=== A frame took 0.277 seconds
[[[178 179 176]
  [177 178 175]
  [176 176 174]
  ...
  [136 143 145]
  [137 143 145]
  [136 142 144]]

 [[176 179 176]
  [175 176 174]
  [176 176 174]
  ...
  [136 143 145]
  [136 142 144]
  [136 142 144]]

 [[176 179 176]
  [173 175 173]
  [173 175 173]
  ...
  [136 142 145]
  [136 142 146

0 person: 44.90
=== A frame took 0.361 seconds
[[[177 180 179]
  [178 179 180]
  [179 179 181]
  ...
  [139 144 147]
  [138 144 148]
  [138 144 148]]

 [[177 180 179]
  [177 179 180]
  [176 178 180]
  ...
  [138 144 147]
  [138 144 148]
  [138 144 148]]

 [[176 178 178]
  [176 178 178]
  [176 178 178]
  ...
  [139 146 147]
  [139 146 147]
  [138 144 146]]

 ...

 [[ 50  64  76]
  [ 50  63  76]
  [ 53  66  78]
  ...
  [156 161 156]
  [154 158 151]
  [154 158 150]]

 [[ 52  65  77]
  [ 55  68  81]
  [ 55  68  81]
  ...
  [109 110 113]
  [123 123 123]
  [116 115 115]]

 [[ 55  68  82]
  [ 57  70  84]
  [ 57  70  83]
  ...
  [112 113 119]
  [114 113 118]
  [113 112 116]]]
0 person: 43.73
=== A frame took 0.283 seconds
[[[182 177 182]
  [182 176 180]
  [181 176 179]
  ...
  [133 144 145]
  [133 144 145]
  [133 144 145]]

 [[180 177 180]
  [179 176 178]
  [179 177 177]
  ...
  [133 144 145]
  [134 145 146]
  [134 145 146]]

 [[180 177 180]
  [180 178 179]
  [179 177 177]
  ...
  [135 145 146

=== A frame took 0.187 seconds
[[[174 178 180]
  [173 177 180]
  [172 175 179]
  ...
  [128 142 147]
  [126 141 146]
  [126 141 146]]

 [[173 177 177]
  [173 177 178]
  [172 176 178]
  ...
  [128 142 147]
  [127 142 147]
  [126 141 146]]

 [[173 177 177]
  [172 176 176]
  [170 175 175]
  ...
  [128 142 147]
  [127 142 147]
  [127 142 147]]

 ...

 [[ 53  62  75]
  [ 53  62  74]
  [ 57  66  77]
  ...
  [160 161 155]
  [160 162 152]
  [158 161 151]]

 [[ 58  65  75]
  [ 60  67  77]
  [ 61  69  77]
  ...
  [129 129 134]
  [125 127 127]
  [118 120 119]]

 [[ 61  68  79]
  [ 63  69  79]
  [ 63  70  78]
  ...
  [119 114 124]
  [123 119 126]
  [122 118 125]]]
=== A frame took 0.212 seconds
[[[172 176 176]
  [172 178 177]
  [169 176 176]
  ...
  [129 142 146]
  [131 143 147]
  [129 142 146]]

 [[171 178 175]
  [170 177 175]
  [169 176 176]
  ...
  [129 142 146]
  [129 142 146]
  [129 142 146]]

 [[171 178 177]
  [170 176 176]
  [169 176 176]
  ...
  [129 142 146]
  [129 142 146]
  [129 142 146

[[[181 179 179]
  [179 178 179]
  [178 178 180]
  ...
  [139 142 149]
  [139 142 149]
  [140 143 150]]

 [[179 180 178]
  [178 178 177]
  [177 177 177]
  ...
  [137 142 149]
  [137 142 149]
  [137 142 149]]

 [[180 181 179]
  [178 178 177]
  [177 177 177]
  ...
  [137 142 149]
  [136 141 147]
  [136 141 147]]

 ...

 [[ 49  62  77]
  [ 50  63  78]
  [ 50  63  78]
  ...
  [162 163 158]
  [162 163 158]
  [163 164 159]]

 [[ 52  65  77]
  [ 52  65  77]
  [ 52  65  77]
  ...
  [120 121 126]
  [120 121 126]
  [118 119 124]]

 [[ 54  67  79]
  [ 54  67  79]
  [ 54  67  79]
  ...
  [113 114 125]
  [114 114 125]
  [113 113 124]]]
=== A frame took 0.179 seconds
[[[175 177 179]
  [175 177 179]
  [175 177 179]
  ...
  [138 141 147]
  [135 139 145]
  [138 141 147]]

 [[175 177 177]
  [176 178 179]
  [175 177 179]
  ...
  [136 140 144]
  [135 140 144]
  [136 142 146]]

 [[176 178 178]
  [176 178 178]
  [175 177 177]
  ...
  [136 142 146]
  [136 142 146]
  [137 143 147]]

 ...

 [[ 51  62  75]
  [ 5

...

0 person: 51.61
=== A frame took 0.324 seconds
[[[176 178 178]
  [176 178 178]
  [176 178 178]
  ...
  [133 140 142]
  [132 138 142]
  [132 138 142]]

 [[177 180 178]
  [176 179 176]
  [176 179 176]
  ...
  [130 140 142]
  [130 139 140]
  [130 139 140]]

 [[177 180 178]
  [176 179 176]
  [176 179 176]
  ...
  [129 139 140]
  [130 139 140]
  [130 139 140]]

 ...

 [[ 49  58  68]
  [ 48  58  69]
  [ 47  57  70]
  ...
  [161 160 162]
  [155 155 155]
  [153 153 152]]

 [[ 49  58  68]
  [ 50  59  71]
  [ 50  58  71]
  ...
  [135 133 138]
  [145 142 147]
  [148 144 149]]

 [[ 50  59  70]
  [ 51  60  72]
  [ 51  60  72]
  ...
  [106 102 109]
  [110 103 113]
  [114 107 117]]]
0 person: 51.60
=== A frame took 0.289 seconds
[[[178 178 178]
  [177 177 178]
  [177 176 179]
  ...
  [133 141 143]
  [131 140 142]
  [131 140 142]]

 [[177 180 179]
  [176 178 178]
  [175 177 177]
  ...
  [132 141 143]
  [131 140 142]
  [132 141 143]]

 [[177 180 179]
  [176 178 178]
  [175 177 177]
  ...
  [130 141 140

0 person: 52.79
=== A frame took 0.304 seconds
[[[183 181 179]
  [181 179 178]
  [180 178 178]
  ...
  [128 141 146]
  [128 141 146]
  [127 139 145]]

 [[180 181 179]
  [179 180 177]
  [179 180 178]
  ...
  [129 139 144]
  [130 141 144]
  [129 140 143]]

 [[180 180 180]
  [180 180 180]
  [179 179 179]
  ...
  [129 140 143]
  [130 141 144]
  [129 140 143]]

 ...

 [[ 49  57  70]
  [ 48  57  70]
  [ 47  57  70]
  ...
  [155 158 155]
  [150 154 149]
  [149 153 148]]

 [[ 50  59  70]
  [ 49  59  69]
  [ 48  59  70]
  ...
  [139 139 139]
  [144 145 143]
  [144 145 142]]

 [[ 50  62  70]
  [ 50  62  70]
  [ 51  63  71]
  ...
  [103 103 107]
  [108 107 112]
  [112 111 115]]]
0 person: 60.41
=== A frame took 0.287 seconds
[[[177 180 179]
  [177 178 178]
  [177 177 177]
  ...
  [133 140 142]
  [135 140 144]
  [133 139 143]]

 [[177 180 179]
  [177 179 179]
  [176 178 178]
  ...
  [133 140 142]
  [133 139 143]
  [133 139 143]]

 [[180 182 182]
  [177 179 179]
  [177 180 179]
  ...
  [133 139 143

0 person: 48.94
=== A frame took 0.264 seconds
[[[181 182 185]
  [180 181 183]
  [180 180 182]
  ...
  [134 143 149]
  [131 142 147]
  [131 142 147]]

 [[181 183 183]
  [178 181 180]
  [177 180 179]
  ...
  [131 142 147]
  [131 142 147]
  [131 141 148]]

 [[181 183 183]
  [178 181 180]
  [177 180 179]
  ...
  [131 142 147]
  [130 140 146]
  [130 140 146]]

 ...

 [[ 48  61  74]
  [ 47  61  74]
  [ 46  61  74]
  ...
  [159 157 157]
  [155 152 153]
  [153 151 151]]

 [[ 48  61  74]
  [ 48  62  75]
  [ 47  63  75]
  ...
  [144 139 141]
  [151 146 147]
  [154 150 150]]

 [[ 48  61  74]
  [ 48  62  75]
  [ 47  63  75]
  ...
  [110 104 111]
  [116 107 115]
  [118 110 117]]]
0 person: 52.47
=== A frame took 0.270 seconds
[[[185 182 185]
  [184 181 183]
  [183 180 182]
  ...
  [136 140 146]
  [137 140 146]
  [138 141 147]]

 [[183 183 183]
  [183 183 183]
  [180 180 180]
  ...
  [135 140 144]
  [133 139 143]
  [135 140 144]]

 [[184 184 184]
  [184 184 184]
  [182 182 182]
  ...
  [133 141 144

0 person: 46.96
=== A frame took 0.291 seconds
[[[176 177 172]
  [175 176 171]
  [175 176 171]
  ...
  [125 137 143]
  [125 137 143]
  [124 136 142]]

 [[174 177 172]
  [174 176 171]
  [175 176 171]
  ...
  [123 138 141]
  [123 138 141]
  [121 137 140]]

 [[175 178 173]
  [173 176 171]
  [173 176 171]
  ...
  [124 139 142]
  [124 139 142]
  [123 138 141]]

 ...

 [[ 36  44  57]
  [ 33  43  54]
  [ 33  44  54]
  ...
  [153 154 149]
  [150 151 147]
  [149 150 145]]

 [[ 33  44  56]
  [ 32  44  56]
  [ 31  44  56]
  ...
  [143 139 137]
  [144 140 138]
  [145 141 139]]

 [[ 33  44  56]
  [ 31  44  56]
  [ 29  44  56]
  ...
  [111 104 107]
  [113 106 109]
  [115 107 110]]]
0 person: 50.67
=== A frame took 0.289 seconds
[[[174 176 176]
  [174 176 176]
  [173 175 175]
  ...
  [133 139 145]
  [131 139 145]
  [130 138 144]]

 [[174 177 174]
  [174 177 174]
  [174 177 174]
  ...
  [131 138 142]
  [130 138 142]
  [130 138 142]]

 [[175 178 175]
  [174 177 174]
  [174 177 174]
  ...
  [129 137 141

0 person: 59.87
=== A frame took 0.287 seconds
[[[175 177 177]
  [173 175 175]
  [173 175 175]
  ...
  [123 136 142]
  [124 136 142]
  [124 136 142]]

 [[175 178 175]
  [173 177 174]
  [170 176 173]
  ...
  [125 136 141]
  [124 136 142]
  [123 136 142]]

 [[176 179 176]
  [174 178 175]
  [172 177 174]
  ...
  [125 135 140]
  [125 135 140]
  [126 136 142]]

 ...

 [[ 41  52  64]
  [ 41  52  64]
  [ 40  50  63]
  ...
  [157 157 153]
  [151 152 148]
  [150 151 146]]

 [[ 43  54  67]
  [ 43  54  66]
  [ 42  53  64]
  ...
  [142 137 136]
  [147 143 141]
  [149 145 142]]

 [[ 47  55  68]
  [ 45  55  67]
  [ 44  56  66]
  ...
  [107 101 106]
  [114 108 113]
  [117 111 117]]]
0 person: 51.74
=== A frame took 0.274 seconds
[[[172 177 174]
  [170 176 173]
  [170 176 173]
  ...
  [121 136 143]
  [123 137 145]
  [123 137 145]]

 [[172 177 172]
  [170 176 171]
  [169 175 170]
  ...
  [121 136 142]
  [121 136 142]
  [123 137 143]]

 [[173 178 175]
  [170 176 173]
  [168 173 171]
  ...
  [121 136 142